In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

In [2]:
cc_df = pd.read_csv('../data/fraud.csv')

In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [5]:
# cc_df의 컬럼명과 자료형을 확인합니다.

cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [6]:
#city별로 mean,std구하고 그 값으로 각 데이터별 z-score 구하기 
#city별로 대도시냐 소도시냐에 따라서 고객들 씀씀이, 물가에 차이가 있을수도 있겠다는 생각. 
city_info = cc_df.groupby('city')['amt'].agg(['mean','std']).reset_index()

In [7]:
city_info

,city,mean,std
0,Alpharetta,50.526703,174.931304
1,Armagh,55.768830,128.260109
2,Bagley,56.593620,112.742183
3,Baton Rouge,90.747123,165.470881
4,Bauxite,90.248648,130.498785
...,...,...,...
116,Westport,70.369762,75.956906
117,Wetmore,60.868976,259.283778
118,Whaleyville,56.593901,125.105931
119,Wilmington,87.442772,129.935554


In [8]:
city_info.to_pickle('./city_info.pkl')

In [9]:
cc_df = cc_df.merge(city_info, on='city', how='left')

In [10]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,mean,std
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,56.078113,159.201852
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0,59.858059,132.138802
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0,87.328067,113.454416
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0,64.317839,174.739042
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0,90.747123,165.470881


In [11]:
cc_df['city_z'] = (cc_df['amt']-cc_df['mean'])/cc_df['std']

In [12]:
#정상 결제일 경우와 사기인 경우의 city_zscore 값이 거의 3정도 차이남. 이 정도면 유의미한 변수라고 볼 수 있을 것 같음. 
print(cc_df.groupby('is_fraud')['city_z'].mean())

is_fraud
0   -0.007404
1    2.915620
Name: city_z, dtype: float64


In [13]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

In [14]:
cc_df

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,city_z
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,0.321302
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0,-0.114637
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0,-0.361802
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0,-0.001304
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0,-0.393828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,fraud_Botsford and Sons,home,134.26,Rebecca,Erickson,F,594 Berry Lights Apt. 392,Wilmington,NC,28405,34.2651,-77.8670,186140,English as a second language teacher,1983-02-08,fc860b0d1f89b0b068c9c8db27b6bcc5,1388534208,34.853497,-78.664158,0,0.360311
491130,2020-12-31 23:56:57,213112402583773,"fraud_Baumbach, Hodkiewicz and Walsh",shopping_pos,25.49,Ana,Howell,F,4664 Sanchez Common Suite 930,Bradley,SC,29819,34.0326,-82.2027,1523,Research scientist (physical sciences),1984-06-03,0f0c38fe781b317f733b845c0d6ba448,1388534217,35.008839,-81.475156,0,-0.173577
491131,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,TX,77566,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0,0.364619
491132,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,WA,99323,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0,-0.015209


In [15]:
# cc_df의 컬럼별 통계량을 확인합니다.

cc_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,city_z
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000,4.911340e+05
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533,2.083304e-18
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264,9.998778e-01
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000,-9.227851e-01
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000,-3.593671e-01
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000,-1.795191e-01
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000,9.523912e-02
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000,6.342123e+01


In [16]:
cc_df.head(3)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,city_z
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,0.321302
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0,-0.114637
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0,-0.361802


In [17]:
# merchant 컬럼에 포함된 값이 몇 종류인지 확인합니다.

cc_df['merchant'].nunique()

693

In [18]:
# Q. job 컬럼에 포함된 값이 몇 종류인지 확인합니다.
cc_df['merchant'].nunique()

693

In [19]:
# Q. cc_num 컬럼에 포함된 값이 몇 종류인지 확인합니다.
cc_df['cc_num'].nunique()

124

In [20]:
# 불필요한 컬럼들을 제거합니다.

cc_df.drop(['merchant','first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)

In [21]:
# cc_num 컬럼의 값을 기준으로 정렬합니다.

cc_df.sort_values('cc_num')

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z
378075,2020-08-05 17:03:19,503874407318,shopping_pos,7.77,M,29.5894,-98.5201,1595797,1975-12-28,30.014539,-98.426767,0,-0.195790
230588,2019-12-20 22:21:36,503874407318,health_fitness,72.06,M,29.5894,-98.5201,1595797,1975-12-28,29.260487,-97.961746,0,0.050666
421413,2020-10-10 12:39:32,503874407318,misc_pos,4.78,M,29.5894,-98.5201,1595797,1975-12-28,29.116928,-99.258936,0,-0.207252
468378,2020-12-13 15:55:44,503874407318,kids_pets,84.56,M,29.5894,-98.5201,1595797,1975-12-28,29.994537,-98.441297,0,0.098585
345085,2020-06-22 23:52:06,503874407318,entertainment,24.33,M,29.5894,-98.5201,1595797,1975-12-28,29.277168,-98.239289,0,-0.132307
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53631,2019-04-14 16:57:31,4956828990005111019,entertainment,27.41,M,40.6747,-74.2239,124967,1980-12-21,39.888460,-74.151440,0,-0.245560
485223,2020-12-27 14:14:40,4956828990005111019,home,28.52,M,40.6747,-74.2239,124967,1980-12-21,41.068207,-75.015316,0,-0.237160
264968,2020-02-15 13:32:48,4956828990005111019,shopping_pos,2.95,M,40.6747,-74.2239,124967,1980-12-21,41.435438,-73.839008,0,-0.430669
63475,2019-04-30 17:45:09,4956828990005111019,shopping_pos,7.28,M,40.6747,-74.2239,124967,1980-12-21,41.149072,-75.143152,0,-0.397900


In [22]:
# z-score 계산을 위한 실습용 데이터입니다.

temp = pd.DataFrame({'a': [10,20,30,20,10,200], 'b': [100,300,200,150,250,200], 'c': [10, 500, 20, 250, 25, 200]})

In [23]:
temp

,a,b,c
0,10,100,10
1,20,300,500
2,30,200,20
3,20,150,250
4,10,250,25
5,200,200,200


In [24]:
# temp 데이터의 컬럼별 평균을 계산합니다.

temp.mean()

a     48.333333
b    200.000000
c    167.500000
dtype: float64

In [25]:
# temp 데이터의 컬럼별 표준편차를 계산합니다.

temp.std()

a     74.677083
b     70.710678
c    192.503247
dtype: float64

In [26]:
# 각각의 데이터에 대해 a 컬럼의 z-score를 계산합니다. 
# (data - mean) / std

(temp['a'] - 48.33) / 74.67

0   -0.513325
1   -0.379403
2   -0.245480
3   -0.379403
4   -0.513325
5    2.031204
Name: a, dtype: float64

In [27]:
# Q. 각각의 데이터에 대해 b 컬럼의 z-score를 계산합니다. 

(temp['b'] - 200) / 70.71

0   -1.414227
1    1.414227
2    0.000000
3   -0.707114
4    0.707114
5    0.000000
Name: b, dtype: float64

In [28]:
# Q. 각각의 데이터에 대해 c 컬럼의 z-score를 계산합니다. 


(temp['c'] - 167.50) / 192.50

0   -0.818182
1    1.727273
2   -0.766234
3    0.428571
4   -0.740260
5    0.168831
Name: c, dtype: float64

In [29]:
# cc_num 컬럼의 값마다 데이터 개수를 계산합니다.

cc_df['cc_num'].value_counts()

6538441737335434    4392
30270432095985      4392
4364010865167176    4386
4642255475285942    4386
6538891242532018    4386
                    ... 
36913587729122      3641
4681601008538160    3638
30551643947183      3638
4005676619255478    3638
3511378610369890    3628
Name: cc_num, Length: 124, dtype: int64

In [30]:
# cc_num 컬럼의 값마다 amt의 평균, 표준편차를 계산하여 amt_info에 저장합니다.

amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean','std']).reset_index()

In [31]:
amt_info.to_pickle('./amt_info.pkl')

In [32]:
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')

In [33]:
cc_df

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,mean,std
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,56.078113,159.201852
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,59.858059,132.138802
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,87.328067,113.454416
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,64.317839,174.739042
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,90.747123,165.470881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,87.442772,129.935554
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,58.181297,188.339282
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,50.452289,168.361122
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,88.704297,119.948793


In [34]:
# Q. 결제금액(amt)의 z-score를 계산하여 amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean'])/cc_df['std'] 

In [35]:
cc_df['amt_z']

0         0.321302
1        -0.114637
2        -0.361802
3        -0.001304
4        -0.393828
            ...   
491129    0.360311
491130   -0.173577
491131    0.364619
491132   -0.015209
491133   -0.152588
Name: amt_z, Length: 491134, dtype: float64

In [36]:
# 사기 거래 데이터(is_fraud=1)를 확인합니다.

cc_df[cc_df['is_fraud'] == 1]

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,mean,std,amt_z
4794,2019-01-12 00:59:01,581686439828,gas_transport,11.73,M,41.5205,-80.0573,5507,1973-07-28,41.947427,-79.796264,1,-0.312731,58.578675,149.804992,-0.312731
4816,2019-01-12 03:48:07,581686439828,grocery_pos,328.68,M,41.5205,-80.0573,5507,1973-07-28,42.148618,-79.398595,1,1.803020,58.578675,149.804992,1.803020
4979,2019-01-12 15:46:10,581686439828,food_dining,120.58,M,41.5205,-80.0573,5507,1973-07-28,42.470024,-80.126576,1,0.413880,58.578675,149.804992,0.413880
5073,2019-01-12 19:53:59,581686439828,shopping_net,1081.35,M,41.5205,-80.0573,5507,1973-07-28,42.455406,-79.521640,1,6.827351,58.578675,149.804992,6.827351
5124,2019-01-12 22:44:05,581686439828,shopping_net,776.70,M,41.5205,-80.0573,5507,1973-07-28,40.680209,-79.099101,1,4.793708,58.578675,149.804992,4.793708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477832,2020-12-21 02:21:41,4716561796955522,grocery_pos,358.24,F,48.2777,-112.8456,743,1972-05-04,47.526202,-113.643313,1,2.880911,52.537867,106.113023,2.880911
477847,2020-12-21 02:36:03,4716561796955522,shopping_net,859.12,F,48.2777,-112.8456,743,1972-05-04,48.272348,-112.328075,1,7.601161,52.537867,106.113023,7.601161
479296,2020-12-21 22:38:38,4716561796955522,home,209.84,F,48.2777,-112.8456,743,1972-05-04,49.173669,-112.698767,1,1.482402,52.537867,106.113023,1.482402
479305,2020-12-21 22:42:11,4716561796955522,food_dining,123.58,F,48.2777,-112.8456,743,1972-05-04,48.913048,-113.214921,1,0.669495,52.537867,106.113023,0.669495


In [37]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

cc_df.drop(['mean','std'], axis = 1, inplace = True)

In [38]:
# cc_num, category 컬럼의 값마다 결제금액(amt)의 평균, 표준편차를 계산하여 cat_info에 저장합니다.

cat_info = cc_df.groupby(['cc_num','category'])['amt'].agg(['mean','std']).reset_index()

In [39]:
cat_info.to_pickle('./cat_info.pkl')

In [40]:
# Q. cc_num 컬럼을 기준으로, cc_df와 cat_info 데이터를 합쳐서 cc_df에 저장합니다.
# (left merge를 수행합니다.)

cc_df = cc_df.merge(cat_info, on=['cc_num','category'], how='left')

In [41]:
# Q. 결제금액(amt)의 z-score를 계산하여 cat_amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean'])/cc_df['std']

In [42]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

cc_df.drop(['mean','std'], axis =1 , inplace = True)

In [43]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,amt_z,cat_amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.321302,0.317631
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.114637,-0.652060
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.361802,-0.723328
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,-0.001304,0.211780
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.393828,-0.575698


In [44]:
# Q. trans_date_trans_time 컬럼에서 시간 값을 추출하여 hour 컬럼에 저장합니다.
# (힌트: datetime 자료형을 이용합니다.)
cc_df['trans_date_trans_time'] = pd.to_datetime(cc_df['trans_date_trans_time'])

In [45]:
cc_df['hour'] = cc_df['trans_date_trans_time'].dt.hour

In [46]:
# 결제시간을 morning, afternoon, night, evening으로 분류하기 위해 함수를 정의합니다.

def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

In [47]:
# Q. hour 컬럼에 hour_func 함수를 적용한 값을 hour_cat 컬럼에 저장합니다.
cc_df['hour_cat'] = cc_df['hour'].apply(lambda x:hour_func(x))

In [48]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,amt_z,cat_amt_z,hour,hour_cat
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.321302,0.317631,0,evening
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.114637,-0.652060,0,evening
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.361802,-0.723328,0,evening
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,-0.001304,0.211780,0,evening
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.393828,-0.575698,0,evening


In [49]:
# Q. 시간대별로 데이터 수를 계산하여 출력합니다.
cc_df.groupby('hour_cat').count()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,amt_z,cat_amt_z,hour
hour_cat,,,,,,,,,,,,,,,,
afternoon,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801,176801
evening,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662,98662
morning,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974,68974
night,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697,146697


In [50]:
# cc_num 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 all_cnt 변수로 저장합니다.

all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

In [51]:
# cc_num, hour_cat 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 hour_cnt 변수로 저장합니다.

hour_cnt = cc_df.groupby(['cc_num','hour_cat'])['amt'].count().reset_index()

In [52]:
all_cnt.head()

,cc_num,amt
0,503874407318,3655
1,567868110212,3644
2,571365235126,4374
3,581686439828,3653
4,630423337322,4362


In [53]:
hour_cnt.head()

,cc_num,hour_cat,amt
0,503874407318,afternoon,1280
1,503874407318,evening,737
2,503874407318,morning,558
3,503874407318,night,1080
4,567868110212,afternoon,1228


In [54]:
# Q. cc_num 컬럼을 기준으로 hour_cnt와 all_cnt를 합칩니다.
# (hour_cnt에서 left merge를 수행합니다.)

hour_cnt= hour_cnt.merge(all_cnt, on='cc_num', how='left')

In [55]:
# Q. amt_x와 amt_y 컬럼의 이름을 각각 hour_cnt, total_cnt로 변경합니다.

hour_cnt.rename({'amt_x': 'hour_cnt', 'amt_y': 'total_cnt'}, axis=1, inplace=True)

In [56]:
hour_cnt.head()

,cc_num,hour_cat,hour_cnt,total_cnt
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [57]:
# Q. 전체 거래 건수(total_cnt)에 대한 시간대별 거래 건수(hour_cnt)의 비율을 계산하여 
# hour_perc 컬럼에 저장합니다.

hour_cnt['hour_perc'] = hour_cnt['hour_cnt'] / hour_cnt['total_cnt']

In [58]:
hour_cnt.head(10)

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
0,503874407318,afternoon,1280,3655,0.350205
1,503874407318,evening,737,3655,0.201642
2,503874407318,morning,558,3655,0.152668
3,503874407318,night,1080,3655,0.295486
4,567868110212,afternoon,1228,3644,0.336992
5,567868110212,evening,820,3644,0.225027
6,567868110212,morning,529,3644,0.145170
7,567868110212,night,1067,3644,0.292810
8,571365235126,afternoon,1523,4374,0.348194
9,571365235126,evening,943,4374,0.215592


In [59]:
hour_cnt.tail(10)

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
486,4861310130652566408,morning,529,3643,0.145210
487,4861310130652566408,night,1033,3643,0.283558
488,4906628655840914250,afternoon,1282,3655,0.350752
489,4906628655840914250,evening,800,3655,0.218878
490,4906628655840914250,morning,500,3655,0.136799
491,4906628655840914250,night,1073,3655,0.293570
492,4956828990005111019,afternoon,1277,3657,0.349193
493,4956828990005111019,evening,784,3657,0.214383
494,4956828990005111019,morning,558,3657,0.152584
495,4956828990005111019,night,1038,3657,0.283839


In [60]:
hour_cnt = hour_cnt[['cc_num','hour_cat','hour_perc']]

In [61]:
hour_cnt.to_pickle('./hour_cnt.pkl')

In [62]:
# Q. cc_num, hour_cat 컬럼을 기준으로 cc_df와 hour_cnt 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

cc_df = cc_df.merge(hour_cnt, on=['cc_num', 'hour_cat'], how='left')

In [63]:
# 시간 관련 feature 분석이 완료되었으므로 불필요해진 컬럼들을 제거합니다.

cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis =1 , inplace = True)

In [64]:
# 거리 계산에 필요한 geopy 라이브러리를 설치합니다.

!pip install geopy

In [65]:
from geopy.distance import distance

In [66]:
# 이렇게 위도와 경도 값으로 두 지점 사이의 거리를 구할 수 있습니다.

distance((48.8878, -118.2105), (49.159047, -118.186462)).km

30.216618410409005

In [67]:
# 고객의 위치와 상점의 위치 사이의 거리를 계산하여 distance 컬럼에 저장합니다.
# (모든 데이터가 계산되므로 실행 시간이 오래 걸릴 수 있습니다.)

cc_df['distance'] = cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)

In [68]:
from datetime import datetime

In [69]:
# 데이터 10000개만 계산해도 시간이 이렇게 걸립니다.

start_time =  datetime.now()
cc_df.head(10000).apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
datetime.now() - start_time

datetime.timedelta(seconds=1, microseconds=890324)

In [70]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,amt_z,cat_amt_z,hour_perc,distance
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.321302,0.317631,0.196470,30.216618
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.114637,-0.652060,0.214383,84.714605
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.361802,-0.723328,0.217252,67.768167
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,-0.001304,0.211780,0.213600,22.322745
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.393828,-0.575698,0.202882,79.398244


In [71]:
dist_info = cc_df.groupby(['cc_num'])['distance'].agg(['mean', 'std']).reset_index()

In [72]:
dist_info

,cc_num,mean,std
0,503874407318,79.746738,29.741044
1,567868110212,79.356969,30.524201
2,571365235126,73.651715,28.863007
3,581686439828,74.428143,28.605102
4,630423337322,71.656621,28.090646
...,...,...,...
119,4792627764422477317,75.499945,29.146824
120,4797297220948468262,75.630035,29.148428
121,4861310130652566408,78.977581,29.793035
122,4906628655840914250,81.148111,30.293827


In [73]:
cc_df = cc_df.merge(dist_info, on='cc_num', how='left')

In [74]:
#거리의 z-score
cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean'])/cc_df['std']

In [75]:
cc_df

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,city_z,amt_z,cat_amt_z,hour_perc,distance,mean,std,dist_z
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031,0.345370
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606,-0.272693
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493,-1.824486
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117,0.002138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,0.360311,1.450980,0.209201,98.043837,77.251486,29.270739,0.710346
491130,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.173577,-0.128877,0.207534,127.240424,77.491795,29.899826,1.663843
491131,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,0.364619,1.163822,0.160339,100.023736,79.172599,30.051452,0.693848
491132,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,-0.015209,0.440137,0.218022,80.887812,72.656420,28.552151,0.288293


In [76]:
cc_df.drop(['lat', 'long', 'merch_lat', 'merch_long', 'mean', 'std'], axis=1, inplace= True)

In [77]:
#생년월일에서 year만 추출
cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

In [78]:
#원핫인코딩
cc_df= pd.get_dummies(cc_df, drop_first=True)

In [79]:
cc_df.head()

,cc_num,amt,city_pop,dob,is_fraud,city_z,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,630423337322,107.23,149,1978,0,0.321302,0.321302,0.317631,0.196470,30.216618,-1.475224,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,4956828990005111019,44.71,124967,1980,0,-0.114637,-0.114637,-0.652060,0.214383,84.714605,0.345370,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,180048185037117,46.28,71485,1974,0,-0.361802,-0.361802,-0.723328,0.217252,67.768167,-0.272693,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,374930071163758,64.09,31515,1971,0,-0.001304,-0.001304,0.211780,0.213600,22.322745,-1.824486,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,2712209726293386,25.58,378909,1977,0,-0.393828,-0.393828,-0.575698,0.202882,79.398244,0.002138,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [80]:
cc_df.drop('cc_num', axis=1, inplace=True)

In [81]:
cc_df

,amt,city_pop,dob,is_fraud,city_z,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,107.23,149,1978,0,0.321302,0.321302,0.317631,0.196470,30.216618,-1.475224,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,44.71,124967,1980,0,-0.114637,-0.114637,-0.652060,0.214383,84.714605,0.345370,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,46.28,71485,1974,0,-0.361802,-0.361802,-0.723328,0.217252,67.768167,-0.272693,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,64.09,31515,1971,0,-0.001304,-0.001304,0.211780,0.213600,22.322745,-1.824486,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,25.58,378909,1977,0,-0.393828,-0.393828,-0.575698,0.202882,79.398244,0.002138,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,134.26,186140,1983,0,0.360311,0.360311,1.450980,0.209201,98.043837,0.710346,0,0,0,0,0,1,0,0,0,0,0,0,0,0
491130,25.49,1523,1984,0,-0.173577,-0.173577,-0.128877,0.207534,127.240424,1.663843,0,0,0,0,0,0,0,0,0,0,0,1,0,0
491131,111.84,28739,1999,0,0.364619,0.364619,1.163822,0.160339,100.023736,0.693848,0,0,0,0,0,0,1,0,0,0,0,0,0,1
491132,86.88,3684,1981,0,-0.015209,-0.015209,0.440137,0.218022,80.887812,0.288293,0,0,0,0,0,0,1,0,0,0,0,0,0,0
